In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Problem Statement

**Context:** The Gurugram-based company ‘FlipItNews’ aims to revolutionize the way Indians perceive finance, business, and capital market investment, by giving it a boost through artificial intelligence (AI) and machine learning (ML). They’re on a mission to reinvent financial literacy for Indians, where financial awareness is driven by smart information discovery and engagement with peers. Through their smart content discovery and contextual engagement, the company is simplifying business, finance, and investment for millennials and first-time investors.

**Objectives:** The goal of this project is to use a bunch of news articles extracted from the companies’ internal database and categorize them into several categories like politics, technology, sports, business and entertainment based on their content. Use natural language processing and create & compare at least three different models.

# Attribute Information:
  * Article
  * Category

# Approoach
1. Importing libraries
2. Loading Data set
        * Reading the data file
3. Data Exploration
      * Shape of Dataset
      * News articles per category
4. Text Processing
      * Removing the non letters
      * Tokenizing the text
      * Removing stopwords
      * Lemmatization
5. Data Transformation
      * Encoding the target variable
      * TF-IDF
      * Train-Test split
6. Model Training & Evalution
      *  Simple Approach
            * MultinomialNB Naive Bayes
      *  Classical ML
            * Decision Tree
            * Random Forest
      *  NLP models
            * LSTM
            * Simple RNN
            * GRU

In [ ]:
# Importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import category_encoders as ce

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier


from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score 



In [ ]:
!pip install gdown


In [ ]:
!gdown --id 1I3-pQFzbSufhpMrUKAROBLGULXcWiB9u

In [ ]:
df = pd.read_csv("/kaggle/working/flipitnews-data.csv")
df.head()

In [ ]:
print("number of rows", df.shape[0])

In [ ]:
plt.figure(figsize = (8,5))
sns.countplot(data = df, x = "Category", palette = "Blues")

In [ ]:
df["Article"][0]

In [ ]:
#Encoding the 'Category' column
encode = ce.OrdinalEncoder(cols = ["Category"])
df = encode.fit_transform(df)

In [ ]:
df.head(10)

# Start with TF-IDF:

TF-IDF: Term Frequency, Inverse Documnet Frequency

In [ ]:
tf_idf = TfidfVectorizer()

X = tf_idf.fit_transform(df["Article"]).toarray()
Y = np.array(df["Category"].values)

In [ ]:
print("Number of features", X.shape[1])

In [ ]:
Y.shape

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X,Y, test_size = 0.25, shuffle = True, stratify =Y)

In [ ]:
print("Number of rows in train data", x_train.shape[0])
print("Number of rows in validation data", x_val.shape[0])

In [ ]:
x_train

In [ ]:
np.unique(x_train[0], return_counts = True)

# Simple Approaches


# MultinomialNB Naive Bayes

In [ ]:
nb = MultinomialNB()
nb.fit(x_train, y_train)

In [ ]:
nb_train_acc = accuracy_score (y_train, nb.predict(x_train))
nb_test_acc = accuracy_score(y_val, nb.predict(x_val))

print("Train Accuracy is", nb_train_acc)
print("Test Accuracy is", nb_test_acc)

In [ ]:
y_pred_nb = nb.predict(x_val)
y_pred_prob_nb = nb.predict_proba(x_val)

roc_auc_score(y_val, y_pred_prob_nb, multi_class = 'ovr')

In [ ]:
precision = precision_score(y_val, y_pred_nb, average = 'weighted')
recall = recall_score(y_val, y_pred_nb, average = "weighted")
f1 = f1_score(y_val, y_pred_nb, average = "weighted")

print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

**Confusion Matrix**

In [ ]:
cm = confusion_matrix(y_val, y_pred_nb)

plt.figure(figsize = (8,5))
sns.heatmap(cm, annot = True, fmt = 'd', cbar = False)

# Lets try Classical ML algorithems

In [ ]:
def model_train(model):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_val)
    y_pred_prob = model.predict_proba(x_val)
    return y_pred, y_pred_prob


def model_eval(model, y_pred, y_pred_proba):
    print("---------------")
    train_acc = accuracy_score (y_train, model.predict(x_train))
    test_acc = accuracy_score (y_val, model.predict(x_val))

    print("Train accuracy:", train_acc)
    print("Test accuracy:", test_acc)

    print()

    print("ROC AUC Score:", roc_auc_score(y_val, y_pred_proba, multi_class = 'ovr'))

    print("Precision:", precision_score(y_val, y_pred, average = 'weighted'))
    print("Recall: ", recall_score(y_val, y_pred, average = "weighted"))
          
    print("f1:", f1_score(y_val, y_pred, average = "weighted"))

    print("--------------------------")


# Decision Tree

In [ ]:
dt = DecisionTreeClassifier()

y_pred_dt, y_pred_proba_dt = model_train(dt)

model_eval(dt, y_pred_dt, y_pred_proba_dt)

# Random Forest

In [ ]:
rf = RandomForestClassifier()

y_pred_rf, y_pred_proba_rf = model_train(rf)

model_eval(dt, y_pred_rf, y_pred_proba_rf)


# LSTM

Long Short Term Memory

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, GRU, SimpleRNN
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv("/kaggle/working/flipitnews-data.csv")
df.head()

In [ ]:
# Parameters
max_features = 5000   #keeping only top most frequent words in the data set for Vocabulary
maxlen = 100          #Considring first 100 words for processing or classification
embedding_size = 100  #Size/dimension of each vector(Word to Vector representation)
batch_size = 500
epochs = 15

In [ ]:
def preprocessing_text(df, text_column):
    df[text_column] = df[text_column].apply(lambda x:x.lower())
    return df

df = preprocessing_text(df,"Article")

In [ ]:
tokenizer = Tokenizer(num_words = max_features) #Initializing the tokenizer object, used to convert text into sequence of integers.The num_words parameter specifies the maximum number of words keep in the tokenization process(Most freequent words in the data set)
tokenizer.fit_on_texts(df['Article'])  #Takes list of texts and processes it to create a word index(Map each word in the article column to a unique integer)

sequences = tokenizer.texts_to_sequences(df["Article"]) #This line converts each text in the article column into a sequence of integers)
data = pad_sequences(sequences, maxlen = maxlen) #This line ensures that all the sequences have the same length.If length of document <100 words, it does this by padding shorter sequences with zeros.

In [ ]:
tokenizer.word_index

In [ ]:
data[0] #Considering only 100 words and an unique integer is assigned to ech unique word

In [ ]:
le = LabelEncoder()

labels = le.fit_transform(df["Category"])
labels = tf.keras.utils.to_categorical(labels)

In [ ]:
labels

In [ ]:
#Train-Test split
x_train, x_test, y_train, y_test = train_test_split(data,labels, test_size = 0.25, random_state = 42)

In [ ]:
# Lets download glove enbedding file

# Step 1: Download the ZIP file
!wget https://huggingface.co/stanfordnlp/glove/resolve/main/glove.6B.zip

In [ ]:
# Step 2: Unzip the file
!unzip glove.6B.zip

In [ ]:
# Open the file in read mode (Looking into embedding file to get more understanding)
with open('glove.6B.100d.txt', 'r') as file:
    # Read the entire file content
    content = file.read()

# Print the content
print(content[0:100])


In [ ]:
content[0:500]

In [ ]:
# Load the glove embedding file and create embedding matrix
def load_glove_embeddings(embedding_path, embedding_dim, tokenizer, max_features):
 
  embeddings_index = {}
 
  with open(embedding_path, 'r', encoding='utf8') as f:
    for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
 
 
  limited_word_index = {word:index for word, index in tokenizer.word_index.items() if index < max_features}
  embedding_matrix =  np.zeros((min(max_features + 1, len(limited_word_index) + 1), embedding_dim))
 
  for word, i in limited_word_index.items():
    if i > max_features:
      continue
 
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector
 
  return embedding_matrix

In [ ]:
embedding_matrix = load_glove_embeddings('glove.6B.100d.txt', embedding_size, tokenizer, max_features)

In [ ]:
embedding_matrix.shape

**Model Building**

In [ ]:
model = Sequential([
    Embedding(max_features, embedding_size, weights = [embedding_matrix],
             trainable = False),
    LSTM(100),
    Dense(len(np.unique(df["Category"])), activation ='softmax')
])

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

**Model Training**

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor ='val_loss', patience = 5)

history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_test, y_test), 
         verbose = 2, callbacks =[early_stopping])

**Validation loss Vs no of Epochs**

In [ ]:
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs') 
plt.ylabel('Validation Loss') 
plt.title('Epochs vs Validation Loss')
plt.legend()
plt.show()

**Model Evalution**

In [ ]:
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis = 1)
y_test_argmax = np.argmax(y_test, axis = 1)

print(classification_report(y_test_argmax, y_pred))

# Simple RNN

In [ ]:
# Model building
model_RNN = Sequential([
    Embedding(max_features, embedding_size, weights = [embedding_matrix],
             trainable = False),
    SimpleRNN(100),
    Dense(len(np.unique(df["Category"])), activation ='softmax')
])

# Compiling
model_RNN.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor ='val_loss', patience = 5)

#Model training
model_RNN.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_test, y_test), 
         verbose = 1, callbacks =[early_stopping])


**Model Evalution**

In [ ]:
y_pred = model_RNN.predict(x_test)
y_pred = np.argmax(y_pred, axis = 1)
y_test_argmax = np.argmax(y_test, axis = 1)

print(classification_report(y_test_argmax, y_pred))


# GRU

In [ ]:
# Model building
model_GRU = Sequential([
    Embedding(max_features, embedding_size, weights = [embedding_matrix],
             trainable = False),
    GRU(100),
    Dense(len(np.unique(df["Category"])), activation ='softmax')
])

# Compiling
model_GRU.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor ='val_loss', patience = 5)

#Model training
model_GRU.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_test, y_test), 
         verbose = 1, callbacks =[early_stopping])

**Model Evalution**

In [ ]:
y_pred = model_GRU.predict(x_test)
y_pred = np.argmax(y_pred, axis = 1)
y_test_argmax = np.argmax(y_test, axis = 1)

print(classification_report(y_test_argmax, y_pred))